In [22]:
from astropy.io import fits
import os
import subprocess
import sys
#from os import system
#from os import spawn*
import shutil
import time

In [ ]:
from shutil import move, copystat

def movetree(src, dst, symlinks=False, ignore=None, move_function=move,
             ignore_dangling_symlinks=False):
    """Recursively copy a directory tree.

    The destination directory must not already exist.
    If exception(s) occur, an Error is raised with a list of reasons.

    If the optional symlinks flag is true, symbolic links in the
    source tree result in symbolic links in the destination tree; if
    it is false, the contents of the files pointed to by symbolic
    links are copied. If the file pointed by the symlink doesn't
    exist, an exception will be added in the list of errors raised in
    an Error exception at the end of the copy process.

    You can set the optional ignore_dangling_symlinks flag to true if you
    want to silence this exception. Notice that this has no effect on
    platforms that don't support os.symlink.

    The optional ignore argument is a callable. If given, it
    is called with the `src` parameter, which is the directory
    being visited by copytree(), and `names` which is the list of
    `src` contents, as returned by os.listdir():

        callable(src, names) -> ignored_names

    Since copytree() is called recursively, the callable will be
    called once for each directory that is copied. It returns a
    list of names relative to the `src` directory that should
    not be copied.

    The optional copy_function argument is a callable that will be used
    to copy each file. It will be called with the source path and the
    destination path as arguments. By default, copy2() is used, but any
    function that supports the same signature (like copy()) can be used.

    """
    names = os.listdir(src)
    if ignore is not None:
        ignored_names = ignore(src, names)
    else:
        ignored_names = set()

    os.makedirs(dst)
    errors = []
    for name in names:
        if name in ignored_names:
            continue
        srcname = os.path.join(src, name)
        dstname = os.path.join(dst, name)
        try:
            if os.path.islink(srcname):
                linkto = os.readlink(srcname)
                if symlinks:
                    os.symlink(linkto, dstname)
                else:
                    # ignore dangling symlink if the flag is on
                    if not os.path.exists(linkto) and ignore_dangling_symlinks:
                        continue
                    # otherwise let the copy occurs. copy2 will raise an error
                    move_function(srcname, dstname)
            elif os.path.isdir(srcname):
                movetree(srcname, dstname, symlinks, ignore, move_function)
            else:
                # Will raise a SpecialFileError for unsupported file types
                move_function(srcname, dstname)
        # catch the Error from the recursive copytree so that we can
        # continue with other files
        except Error as err:
            errors.extend(err.args[0])
        except EnvironmentError as why:
            errors.append((srcname, dstname, str(why)))
    try:
        copystat(src, dst) #Noss: this can stay
    except OSError as why:
        if WindowsError is not None and isinstance(why, WindowsError):
            # Copying file access times may fail on Windows
            pass
        else:
            errors.append((src, dst, str(why)))
    if errors:
        raise Error(errors)

In [11]:
startTime = time.time()

regressionPath = '/Users/jamie/dev/regression/regress'
execPath = '/Users/jamie/dev/builds/master'
outPath = '/Users/jamie/dev/regression/master/'

print('Path containing test data: "{0}"'.format(regressionPath))
print('Path containing executables: "{0}"'.format(execPath))
print('Path to dump all output: "{0}"'.format(outPath))
print('\n')

cteOnly = True
compareOnTheFly = False
compareOnly = False

#create output dir and/or move old one to outPath + '-old'
try:
    os.mkdir(outPath)
except FileExistsError:
    sys.exit('uh oh, that was a close one! The output path "{0}" already exists, please delete or use another path'.format(outPath))
logPath = os.path.join(outPath, 'logs')
os.mkdir(logPath)

wf3Input = []
acsInput = []
stisInput = []

tree = os.walk(regressionPath, topdown = True, followlinks = False)
for root, dir, files in tree:
    for fname in files:
        if 'raw.fits' in fname and 'raw.fits.svn-base' not in fname:
            fullFilePath = os.path.join(root, fname)
            hdu = fits.open(fullFilePath)
            instrument = hdu[0].header['instrume']
            hdu.close()
            if 'WFC3' in instrument:
                wf3Input.append(fullFilePath)
            elif 'ACS' in instrument:
                acsInput.append(fullFilePath)
            elif 'STIS' in instrument:
                stisInput.append(fullFilePath)

print('{0} acs input files found.'.format(str(len(acsInput))))
print('{0} stis input files found.'.format(str(len(stisInput))))
print('{0} wf3 input files found.\n'.format(str(len(wf3Input))))

print('Time taken to walk directory tree: {0}\n'.format(str(time.time() - startTime)))

if cteOnly:
    print('Processing CTE corrections only:\n')

    wf3CTEInput = []
    for entry in wf3Input:
        hdu = fits.open(entry)
        try:
            cteCorrect = hdu[0].header['PCTECORR']
        except KeyError:
            #print('Missing "PCTECORR" keyword, skipping ' + entry)
            continue
        hdu.close()
        if 'PERFORM' in cteCorrect:
            wf3CTEInput.append(entry)
    
    print('{0} wf3cte input files found.\n'.format(str(len(wf3CTEInput))))
    
    cmd = os.path.join(execPath, 'wf3cte.e')
    if os.path.exists(cmd) == False:
        sys.exit('ERROR: The required executable "{0}" does not exist!'.format(cmd))
    
    #Acquire version
    version = subprocess.run([cmd, "--version"], shell=False, check=False, stdout=subprocess.PIPE, universal_newlines=True).stdout
       
    os.chdir(outPath)
    for entry in wf3CTEInput:
        print('\nProcessing "{0}"...'.format(entry))
        #Run wf3cte on relevant files (one at a ime)
        results = subprocess.run(['time', cmd, "-v", entry], shell=False, check=False, universal_newlines=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        #Now look at results and write them to a log file in outPath
        logFile = os.path.basename(entry) + '.log'
        logFile = os.path.join(logPath, logFile)
        try:
            fid = open(logFile, mode='w')
        except OSError as err:
            print('ERROR: Cannot write log file "{0}" due to:'.format(logFile))
            print('"{0}"'.format(err))
        else:
            fid.write('Input file: "{0}"\n'.format(entry))
            fid.write('Program: "{0}"\n'.format(cmd))
            fid.write('Version: {0}\n'.format(str(version)))
            fid.write('return code:{0}\n\n'.format(str(results.returncode)))
            fid.write('stdout results:\n {0}\n\n'.format(str(results.stdout)))
            fid.write('stderr results:\n {0}\n\n'.format(str(results.stderr)))
            fid.close()
        
        if results.returncode:
            print('Process failed')
        else:
            print('Process succeded')         
            
            #Check each test on the fly
            if compareOnTheFly:
                print('Comparing results...')
                #do comparison with FITSDiff
            
#move generated files to outPath/results (add 'results' so that copytree can be used)
movetree(regressionPath, os.path.join(outPath, 'results'), ignore=shutil.ignore_patterns('*raw.fits'))



print('\nTotal time taken: {0} (seconds)'.format(str(time.time() - startTime)))



Path containing test data: "/Users/jamie/dev/regression/regress"
Path containing executables: "/Users/jamie/dev/builds/master"
Path to dump all output: "/Users/jamie/dev/regression/master/"


24 acs input files found.
4 stis input files found.
490 wf3 input files found.

Time taken to walk directory tree: 27.648819000000003

Processing CTE corrections only:

5 wf3cte input files found.


Processing "/Users/jamie/dev/regression/regress/wf3/uvis/iaao01k8q_raw.fits"...
Process succeded

Processing "/Users/jamie/dev/regression/regress/wf3/uvis/iacr51ohq_raw.fits"...
Process succeded

Processing "/Users/jamie/dev/regression/regress/wf3/uvis/iacr51ojq_raw.fits"...
Process succeded

Processing "/Users/jamie/dev/regression/regress/wf3/uvis/iacr51okq_raw.fits"...
Process succeded

Processing "/Users/jamie/dev/regression/regress/wf3/uvis/iacr51omq_raw.fits"...
Process succeded

Total time taken: 31.387382000000002 (seconds)


In [8]:
rawfile = fits.open('dev/test/cte/wf3/truth/ic0q01ynq_raw.fits')
truthfile = fits.open('dev/test/cte/wf3/truth/ic0q01ynq_rac_tmp.fits')
devfile = fits.open('dev/test/cte/wf3/ic0q01ynq_rac_tmp.fits')

diff = fits.FITSDiff(devfile, truthfile, ignore_keywords=['date'])
diff.identical

pixDiff = fits.ImageDataDiff(rawfile[1].data, truthfile[1].data, tolerance=0.0, numdiffs=10)
print(pixDiff.identical)
pixDiff.diff_pixels
#diff.report()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "//anaconda/envs/astroconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-e565318b6a94>", line 1, in <module>
    rawfile = fits.open('dev/test/cte/wf3/truth/ic0q01ynq_raw.fits')
  File "//anaconda/envs/astroconda/lib/python3.5/site-packages/astropy/io/fits/hdu/hdulist.py", line 166, in fitsopen
    lazy_load_hdus, **kwargs)
  File "//anaconda/envs/astroconda/lib/python3.5/site-packages/astropy/io/fits/hdu/hdulist.py", line 404, in fromfile
    lazy_load_hdus=lazy_load_hdus, **kwargs)
  File "//anaconda/envs/astroconda/lib/python3.5/site-packages/astropy/io/fits/hdu/hdulist.py", line 1015, in _readfrom
    fileobj = _File(fileobj, mode=mode, memmap=memmap, cache=cache)
  File "//anaconda/envs/astroconda/lib/python3.5/site-packages/astropy/utils/decorators.py", line 493, in wrapper
    return function(*args, **kwargs)
  File "//

FileNotFoundError: [Errno 2] No such file or directory: 'dev/test/cte/wf3/truth/ic0q01ynq_raw.fits'

NotADirectoryError: [Errno 20] Not a directory: '/Users/jamie/dev/regression/regress/wf3/uvis/iaao01k8q.tra'

acs
stis
wf3


In [22]:
shutil.copytree(regressionPath, os.path.join(outPath, 'results'), ignore=shutil.ignore_patterns('*raw.fits'))


'/Users/jamie/dev/regression/master/results'

In [34]:
compare = True

masterPath = '/Users/jamie/dev/regression/regress/master'
devPath = '/Users/jamie/dev/regression/regress/dev'

#compare results
if compare:
    masterPath = comparePath
    devPath = outPath
    #check both dir to compare exist
    isMaster = os.path.exists(masterPath)
    isDev = os.path.exists(devPath)
    if isMaster == False and isDev == False
        sys.exit('ERROR: The Paths, "{0}" and "{1}", do not exist to compare')
    if isMaster == False
        sys.exit('ERROR: comparison not possible, "{0}" doesn\'t exist'.format(masterPath))
    if isDev == False
        sys.exit('ERROR: comparison not possible, "{0}" doesn\'t exist'.format(devPath))
        
    